### Live Prediction

In [ ]:
# !pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import cv2
from ultralytics import YOLO

def load_model(model_path="best.pt"):
    """
    Loads a YOLOv8 model from the given path.
    """
    return YOLO(model_path)

def predict_frame(model, frame, conf_thresh=0.25):
    """
    Runs prediction on a single frame and returns annotated frame.
    """
    results = model.predict(source=frame, conf=conf_thresh, verbose=False)

    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cls_id = int(box.cls[0])
            conf = float(box.conf[0])
            label = f"{model.names[cls_id]}: {conf:.2f}"

            # Draw bounding box and label
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    return frame

def run_webcam_detection(model_path="best.pt"):
    """
    Opens webcam and runs YOLOv8 object detection in real-time.
    """
    model = load_model(model_path)
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    print("Press 'q' to exit...")
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        annotated_frame = predict_frame(model, frame)

        cv2.imshow("YOLOv8 Live Detection", annotated_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [ ]:
def find_best_model(base_dir='runs_yolo/'):
    best_paths = list(Path(base_dir).rglob('best.pt'))
    if not best_paths:
        raise FileNotFoundError("No 'best.pt' file found in the 'runs/' directory.")
    
    # Optionally, sort by latest modified time
    best_paths.sort(key=lambda p: p.stat().st_mtime, reverse=True)
    
    print(f"✅ Found best.pt at: {best_paths[0]}")
    return str(best_paths[0])


In [3]:
yolov8 = './runs/train/yolov8'
best_pt_path = find_best_model(yolov8)
run_webcam_detection(best_pt_path)


NameError: name 'find_best_model' is not defined